<a href="https://colab.research.google.com/github/PhilipMathieu/unet-orthoimagery/blob/main/colab_augment_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
!rm -r sample_data
!git clone --single-branch --branch main https://github.com/PhilipMathieu/unet-orthoimagery.git ./repo/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=cba62e96630df972a4fc428ad718f03006555530ef0e636efa62032547cc6031
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
Cloning into './repo'...
remote: Enumerating objects: 109124, done.
remote: Counting objects: 100% (4299/4299), done.
remote: Compressing objects: 100% (2606/2606), done.
remote: Total 109124 (delta 1725), reused 4254 (delta 1692), pack-reused 1048

In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import sys
sys.path.append('./repo/src/')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from dataAugmentation import main

/content/./repo/src/dataAugmentation.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
main("repo/data/Image_Chips_128_overlap_balanced_dem", "AD")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [7]:
import logging
import torch
from train import train_model
from unet.unet_model import UNet

In [9]:
data_dir = "AD/"
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device: {device}')
print(device)

In [ ]:
%%time
model = UNet(n_channels=4, n_classes=1, bilinear=False)
model = model.to(device, memory_format=torch.channels_last)

def train():
    train_model(
        model=model,
        epochs=20,
        device=device,
        data_dir=data_dir,
        batch_size=128,
        learning_rate=1e-3,
        img_scale=1,
        debug=False
    )

try:
    train()
except torch.cuda.OutOfMemoryError: # Giving me syntax error saying '"OutOfMemoryError" is not a valid exception class.'
    logging.error('Detected OutOfMemoryError! '
                    'Enabling checkpointing to reduce memory usage, but this slows down training. '
                    'Consider enabling AMP (--amp) for fast and memory efficient training')
    torch.cuda.empty_cache()
    model.use_checkpointing()
    train()

cuda


  0%|          | 0/2560 [00:00<?, ?it/s]

Epoch 1/20:   0%|          | 0/1708 [00:00<?, ?img/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

...

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Epoch 2/20:   0%|          | 0/1708 [00:00<?, ?img/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

In [11]:
wandb.finish()